In [119]:
### Retención entrada
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime as dt
pd.options.display.max_columns = None
from datetime import timedelta
import glob as glob
import warnings
import os
import jenkspy
import sys
warnings.filterwarnings("ignore", category=FutureWarning)
import zipfile
os.getcwd()
now=dt.datetime.now().strftime('%Y%m%d')
now2=dt.datetime.now().strftime('%Y-%m-%d')
today = dt.date.today()
hoy=now

import glob as glob
def condition():
    if today.strftime("%A")=='Monday':
        return (today - dt.timedelta(days=2)).strftime('%Y-%m-%d')
    else:
        return (today - dt.timedelta(days=1)).strftime('%Y-%m-%d')
yesterday=condition()

### Speech

In [120]:
# speech=pd.read_csv(r"\\10.1.1.7\01 Oficina Planeación y Control\01 Analytics\01 Informes_Analitica\Super_Sherlock_Report\CarpetaOculta\SemTopico_retencion.csv",dtype=str,encoding='latin-1')

In [121]:
# speech=pd.read_csv(r"\\10.1.1.7\01 Oficina Planeación y Control\01 Analytics\01 Informes_Analitica\Super_Sherlock_Report\CarpetaOculta\SemTopico_retencion.csv",dtype=str,encoding='latin-1')
# speech.drop(['Unnamed: 0'],axis=1,inplace=True)
speech=pd.read_csv(r"Y:\01 Informes_Analitica\Super_Sherlock_Report\CarpetaOculta\Speech_Retencion_Viaje.csv",dtype=str,encoding='utf-8',sep=';')


In [122]:
speech['Cat_Sac'].value_counts(dropna=False)

fallas y daños en el servicio          6848
no uso del servicio                    3132
solicitud retiro parcial               2850
motivos económicos                     2330
problemas con traslado del producto    2092
agendamiento                           1279
competencia producto                   1094
duplicidad del servicio                 834
venta o entrega del inmueble            741
cliente inconforme                      343
viaje                                    79
retiro canales premium                   67
Name: Cat_Sac, dtype: int64

In [123]:
speech['ID_llamada'].nunique()

12320

In [124]:
## Lectura de xlsm
# junio=pd.read_excel(r"\\10.1.1.7\01 Oficina Planeación y Control\32-GTR\01 - UNE\UNE - Fidelizacion\Operación 2016\Efectividad\2021\06 Junio\Efectividad + ARPU En curso Junio.xlsm",sheet_name='BD IDC')
# junio.drop(junio.filter(like='Un'),axis=1,inplace=True)
# junio.to_csv(r"C:\Users\scadacat\Desktop\Retención_2\Bds reteción\junio.csv",sep=',',encoding='utf-8',index=False)

### Lectura BD IDC

In [125]:

agosto=pd.read_csv(r"C:\Users\scadacat\Desktop\Retención_2\Bds reteción\agosto.csv",sep=',',encoding='utf-8',dtype=str)
julio=pd.read_csv(r"C:\Users\scadacat\Desktop\Retención_2\Bds reteción\julio.csv",sep=',',encoding='utf-8',dtype=str)
sept=pd.read_csv(r"C:\Users\scadacat\Desktop\Retención_2\Bds reteción\sept.csv",sep=',',encoding='utf-8',dtype=str)
octu=pd.read_csv(r"C:\Users\scadacat\Desktop\Retención_2\Bds reteción\octubre.csv",sep=',',encoding='utf-8',dtype=str)

In [126]:
# octu.drop(octu.filter(like='Un'),axis=1,inplace=True)
# octu.to_csv(r"C:\Users\scadacat\Desktop\Retención_2\Bds reteción\octubre.csv",sep=',',encoding='utf-8',index=False)

In [127]:
data=pd.concat([julio,agosto,sept,octu])
data.reset_index(drop=True,inplace=True)
del agosto, julio,sept,octu

In [128]:
data.reset_index(drop=True,inplace=True)
data['ID_LLAMADA']=data['ID_LLAMADA'].str.strip('_')
data['FE_REGISTRO']=pd.to_datetime(data['FE_REGISTRO'],format ='%Y-%m-%d %H:%M:%S',errors = "coerce")
data.sort_values(['ID_LLAMADA','FE_REGISTRO'],ascending=True,inplace=True)
data=data[data['RESULTADO_SS']!='Ha fallado la conexi&oacute;n al MySQL Local: ']
data=data[data['Agrupación']!='MT']
data=data[(data['Fallidas'].astype(int)+data['Exitosas'].astype(int))>=1]
data[['SERVICIO_1','SERVICIO_2','SERVICIO_3']]=data[['SERVICIO_1','SERVICIO_2','SERVICIO_3']].fillna('')
data['SERVICIOS']=data['SERVICIO_1']+"_"+data['SERVICIO_2']+"_"+data['SERVICIO_3']
data=data[~data['ID_LLAMADA'].duplicated(keep='last')]

### Cleaning

In [129]:
# data[data['ID_LLAMADA']=='00001121041631912511']

In [130]:
dic={'INTERNET_TELEVISION_':'INTERNET_TELEVISION',
    'INTERNET_TELEFONIA_':'INTERNET_TELEFONIA',
    'INTERNET_TELEFONIA_':'INTERNET_TELEFONIA',
    'INTERNET_INTERNET CONMUTADO_TELEFONIA':'INTERNET_TELEFONIA',
    'INTERNET_SERV PRO Y SERV ADMIN_TELEVISION':'INTERNET_TELEVISION',
    'INTERNET_MCAFEE_TELEVISION':'INTERNET_TELEVISION',
    'INTERNET_PRODUCTO POR DEMANDA_TELEFONIA':'INTERNET_TELEFONIA',
    'CONTENIDOS IDIOMAS UNE BERLITZ_INTERNET_TELEFONIA':'INTERNET_TELEFONIA',
    'INTERNET_SERVICIOS Y EQUIPOS_TELEFONIA':'INTERNET_TELEFONIA',
    '__':'',
    'TELEFONIA_TELEVISION_':'TELEFONIA_TELEVISION',
    'TELEVISION__':'TELEVISION',
    'INTERNET_TELEFONIA_TELEFONIA':'INTERNET_TELEFONIA',
    'INTERNET_INTERNET_INTERNET':'INTERNET',
    'INTERNET_INTERNET_':'INTERNET',
    '_MCAFEE_':'',
    'TELEFONIA_TELEFONIA_':'TELEFONIA',
    'INTERNET_INTERNET_TELEVISION':'INTERNET_TELEVISION',
    'TELEFONIA__':'TELEFONIA',
    'INTERNET__':'INTERNET',
    '_MCAFEE_ ':'','BOLSA DINAMICA_TELEFONIA_TELEFONIA':'TELEFONIA_TELEFONIA',
    'INTERNET_MCAFEE_':'INTERNET','TELEVISION_TELEVISION_':'TELEVISION_TELEVISION',
    'MCAFEE_':'',
    'INTERNET_NUMERO UNICO_TELEFONIA':'INTERNET_TELEFONIA',
    'INTERNET_MARIA DAMARIS HINCAPIE ALZATE_TELEFONIA':'INTERNET_TELEFONIA',
    'INTERNET_INTERNET_MCAFEE':'INTERNET_MCAFEE',
    'INTERNET_TELEVISION_TELEVISION':'INTERNET_TELEVISION',
    'INTERNET_INTERNET_TELEFONIA':'INTERNET_TELEFONIA',
    'INTERNET_MCAFEE_TELEFONIA':'INTERNET_TELEFONIA',
    'TELEFONIA_TELEFONIA_TELEVISION':'TELEFONIA_TELEVISION',
    'INTERNET_MCAFEE':'INTERNET',
    'INTERNET_INTERNET CONMUTADO_':'INTERNET',
    'CONSUMOS UNE LD-MOVIL_INTERNET_TELEFONIA':'INTERNET_TELEFONIA',
    '113_INTERNET_':'INTERNET',
    'INTERNET_TELECOM_TELEFONIA':'INTERNET_TELEFONIA',
    '113_INTERNET_TELEFONIA':'INTERNET_TELEFONIA',
    'EL COLOMBIANO_INTERNET_TELEFONIA':'INTERNET_TELEFONIA',
    'TELEVISION_TELEVISION':'TELEVISION','TELEFONIA_TELEFONIA':'TELEFONIA'}
data['SERVICIOS']=data['SERVICIOS'].replace(dic)
data['Efectividad']=(data['Exitosas'].astype(int)/(data['Exitosas'].astype(int)+data['Fallidas'].astype(int)))*100
data['Efectividad']=round(data['Efectividad']).astype(int)
conteo3=data['SERVICIOS'].value_counts().iloc[:7].index.tolist()
data['SERVICIOS_2']=data.apply(
lambda row: row['SERVICIOS']  if (row['SERVICIOS'] in conteo3)
else 'OTROS PRODUCTOS',axis=1)
data['Zona']=np.where(data['DIRECCION'].str.upper().str.contains('RURAL',na=False),'RURAL','URBANA')

In [131]:
################################################               TO                           #######################################

# data[data['ESTADO_ACCION_TO']=='0'][cols.insert(0,'Efectivo').insert(0,'Fallidas').insert(0,'Exitosas')]
col         = 'ESTADO_ACCION_TO'
co2         ='Segmento'
conditions  = [ (data[col]=='Exitosa'),(data[col]=='Fallida'), ((data[col]!='Exitosa')  & (data[col]!='Fallida'))]
choices     = [ "TO", "no TO","Otro caso"]

data["ESTADO_ACCION_TO_2"] = np.select(conditions, choices, default=np.nan)


################################################               TV                           #######################################

col         = 'ESTADO_ACCION_TV'
conditions  = [ (data[col]=='Exitosa'),(data[col]=='Fallida'), ((data[col]!='Exitosa')  & (data[col]!='Fallida'))]
choices     = [ "TV", "no Tv","Otro caso"]

data["ESTADO_ACCION_TV_2"] = np.select(conditions, choices, default=np.nan)

################################################               BA                           #######################################
col         = 'ESTADO_ACCION_BA'
conditions  = [ (data[col]=='Exitosa'),(data[col]=='Fallida'), ((data[col]!='Exitosa')  & (data[col]!='Fallida'))]
choices     = [ "BA", "no BA","Otro caso"]

data["ESTADO_ACCION_BA_2"] = np.select(conditions, choices, default=np.nan)
data['consolidación_accion']=data['ESTADO_ACCION_BA_2']+"_"+data['ESTADO_ACCION_TO_2']+"_"+data['ESTADO_ACCION_TV_2']
dic2={'BA_TO_TV':'Retencion BA_TO_TV','no BA_no TO_no Tv':'No retencion BA_TO_TV',
        'BA_Otro caso_TV':'Retencion BA_TV','BA_TO_Otro caso':'Retencion BA_TO',
        'BA_Otro caso_Otro caso':'Retencion BA','no BA_Otro caso_no Tv':'No retencion BA_TV',
        'no BA_Otro caso_Otro caso':'No retencion BA','no BA_no TO_Otro caso':'No retencion BA_TO',
        'Otro caso_TO_Otro caso':'Retencion TO','Otro caso_no TO_Otro caso':'No retencion TO',
        'Otro caso_Otro caso_TV':'Retencion TV','Otro caso_Otro caso_no Tv':'No retencion TV',
        'Otro caso_TO_TV':'Retencion TO_TV','BA_no TO_TV':'Retencion BA_TV no retencion TO',
        'Otro caso_no TO_no Tv':'No retencion TO_TV','BA_no TO_no Tv':'Retencion BA no retencion TO_TV','BA_TO_no Tv':'Retencion BA_TO no retencion TV',
        'BA_Otro caso_no Tv':'Retencion BA no retencion TV','BA_no TO_Otro caso':'Retencion BA no retencion TO','no BA_TO_Otro caso':'No retencion BA retencion TO',
        'no BA_TO_no Tv':'No retencion BA_TV retencion TO','no BA_TO_TV':'No retencion BA retencion TO_TV','no BA_no TO_TV':'No retencion BA_TO retencion TV',
        'no BA_Otro caso_TV':'No retencion BA Retencion TV','Otro caso_no TO_TV':'No Retencion TO retencion TV','Otro caso_TO_no Tv':'Retencion TO no retencion TV'}
data['accion']=data['consolidación_accion'].replace(dic2).str.upper()
## Formato texto y variable de secuencia-- seguido se queda con el último
data.loc[data['IDENT_CLIENTE'].str.len() < 7, "IDENT_CLIENTE"] = np.nan  
data.dropna(subset=['IDENT_CLIENTE'],inplace=True)     
data['IDENT_CLIENTE']=data['IDENT_CLIENTE'].astype(str)
data['ano']=data['FE_REGISTRO'].dt.year.astype(str)
data['mes']=data['FE_REGISTRO'].dt.month.astype(str)
cant_serv=data.groupby(['IDENT_CLIENTE']).agg({'DIRECCION':'nunique'})\
                                                        .reset_index()\
                                                        .rename(columns={'DIRECCION':'serv_dir'})
data=pd.merge(data,cant_serv,on='IDENT_CLIENTE')
data['serv_dir'][data['serv_dir']==0]=1
data['sequence']=data.sort_values(by=['IDENT_CLIENTE','ano','mes','FE_REGISTRO'], ascending=True).\
                     groupby(['IDENT_CLIENTE','ano','mes']).cumcount()+1

data.drop(data.filter(like='PEDID').columns,axis=1,inplace=True)

############################################################                 EXPORTE                   ###############################################
data.to_csv(r'C:\Users\scadacat\Desktop\Retención_2\consolidado_Bd_efectividad.csv',sep=';',encoding='utf-8',index=True)

C:\Users\scadacat\AppData\Local\Temp/ipykernel_14288/3021754573.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['serv_dir'][data['serv_dir']==0]=1


In [132]:
data.head()

,Día,Hora,Fallidas,Exitosas,Regional,Efectivo,Agrupación,CONTRATO,NOMBRE_CLIENTE,IDENT_CLIENTE,DIRECCION,CICLO,CIUDAD_DPTO,SERVICIO_1,SERV_SUSCRITO_1,PLAN_SERVICIO_1,EST_CONEXION_1,SERVICIO_2,SERV_SUSCRITO_2,PLAN_SERVICIO_2,EST_CONEXION_2,SERVICIO_3,SERV_SUSCRITO_3,PLAN_SERVICIO_3,EST_CONEXION_3,INI_PER_CONSUMO,FIN_PER_CONSUMO,ENTREGA_CAMBIOS,m,N,MOTIVO,RESULTADO_SS,ESTADO_ACCION_TO,ESTADO_ACCION_TV,ESTADO_ACCION_BA,ID_LLAMADA,MT,MOTIVO_MT,FE_REGISTRO,USER_REG,ID_REGISTRO,BENEF_MOVIL,BENEF_TO,RECONFIG_TO,DESCUENTO_TO,BENEF_TV,RECONFIG_TV,DESCUENTO_TV,BENEF_BA,RECONFIG_BA,DESCUENTO_BA,ORIGEN,NOMBRE_CDVS,ESTADO_ACCION_CP,TACTICO_CP,NIVELUNO_APLICADO,OBSERVACIONES,FE_NOTIFICADO,NOTIFICADO,A,B,GRUPO,Tipo de portafolio,Following,Supervisor,Segmento,Nn,Supervisor M,SERVICIOS,Efectividad,SERVICIOS_2,Zona,ESTADO_ACCION_TO_2,ESTADO_ACCION_TV_2,ESTADO_ACCION_BA_2,consolidación_accion,accion,ano,mes,serv_dir,sequence
0,29,15,0,3,NOROCCIDENTE,Retenido,Servicio,14257256,VALENCIA MAZO SARITA,1036654856,RURAL_159021398000000000_159021398000000000,5,LA ESTRELLA-ANTIOQUIA,INTERNET,127148359,5M NET DUO BRON,CONEXION,TELEFONIA,94721713,ILIMITADO LOCAL,CONEXION,TELEVISION,120762065,ESENCIAL DUO,CONEXION,NaN,NaN,Pendiente por definir,Ret. Intencion de Retiro,Mala calidad del Producto,Danos en los servicios,Desfavorable,Exitosa,Exitosa,Exitosa,,NO,NaN,2021-07-29 15:25:44,cramirgo,99992623730,NaN,BCCO (Cliente lo va consultar x Oferta N2),NaN,NaN,BCCO (Cliente lo va consultar x Oferta N2),NaN,NaN,BCCO (Cliente lo va consultar x Oferta N2),NaN,NaN,RETENCION NIVEL2,RETENCION NIVEL2,0,NaN,NaN,Cliente se comunica con intension de retiro in...,NaN,NaN,NaN,4444098,No,6meses,NaN,50,Legend,NaN,NaN,INTERNET_TELEFONIA_TELEVISION,100,INTERNET_TELEFONIA_TELEVISION,RURAL,TO,TV,BA,BA_TO_TV,RETENCION BA_TO_TV,2021,7,1,1
1,6,15,0,3,ORIENTE,Retenido,Servicio,17883032,EDWIN ANDRES MONSALVE ORTEGA,1093770378,RURAL_900680000013988929_CL 45 # 0 - 172 IN 90...,7,BUCARAMANGA-SANTANDER,INTERNET,133460588,Internet 15MB Trio,CONEXION,TELEFONIA,133460592,TO ILIMITADO 1.0 TRIO,CONEXION,TELEVISION,133460590,CLASICA HD TRIO,CONEXION,NaN,NaN,Ciclico SEPTIEMBRE/ Entre 24-AGO-2021 al 26-AG...,Ret. Intencion de Retiro,Mala calidad del Producto,Danos en los servicios,Desfavorable,Exitosa,Exitosa,Exitosa,\t0000113664162560475,NO,NaN,2021-07-06 15:54:51,cramirgo,99992557598,NaN,NaN,Reconfig. Igual de ARPU Recomendador IDC,NaN,NaN,Reconfig. Igual de ARPU Recomendador IDC,NaN,NaN,Reconfig. Igual de ARPU Recomendador IDC,NaN,RETENCION NIVEL2,RETENCION NIVEL2,0,NaN,NaN,Cliente se comunica con intension de retiro ya...,NaN,NaN,4322733,4322818,No,6m,NaN,NaN,Master,NaN,NaN,INTERNET_TELEFONIA_TELEVISION,100,INTERNET_TELEFONIA_TELEVISION,RURAL,TO,TV,BA,BA_TO_TV,RETENCION BA_TO_TV,2021,7,1,1
2,10,16,3,0,ORIENTE,No Retenido,Servicio,17883032,EDWIN ANDRES MONSALVE ORTEGA,1093770378,RURAL_900680000013988929_CL 45 # 0 - 172 IN 90...,7,BUCARAMANGA-SANTANDER,INTERNET,133460588,Internet 15MB Trio,CONEXION,TELEFONIA,133460592,TO ILIMITADO 1.0 TRIO,CONEXION,TELEVISION,133460590,CLASICA HD TRIO,CONEXION,NaN,NaN,Ciclico SEPTIEMBRE/ Entre 24-AGO-2021 al 26-AG...,Ret. Intencion de Retiro,Mala calidad del Producto,Danos en los servicios,Favorable,Fallida,Fallida,Fallida,00001012711625949254,NO,NaN,2021-07-10 16:18:07,ylondlon,99992570817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RETENCION NIVEL2,RETENCION NIVEL2,0,NaN,NaN,Se comunica user manifestando intencion de ret...,NaN,NaN,4346842,4347035,No,6meses,NaN,MARIA ALEJANDRA HERRERA LUGO,Legend,NaN,NaN,INTERNET_TELEFONIA_TELEVISION,0,INTERNET_TELEFONIA_TELEVISION,RURAL,no TO,no Tv,no BA,no BA_no TO_no Tv,NO RETENCION BA_TO_TV,2021,7,1,2
3,26,11,0,1,NOROCCIDENTE,Retenido,Servicio,16876611,JOSE RAUL TABARES GALEANO,1040048597,CL 25 CR 19 -61,4,LA CEJA-ANTIOQUIA,INTERNET,132114742,Internet 60MB Ind,CONEXION,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,Pendiente por definir,Ret. Intencion de Retiro,Mala calidad del Producto,Danos en los servicios,Desfavorable,0,0,Exitosa,\t477

In [133]:
speech.rename(columns={'ID_llamada':'UcidXSegmento'},inplace=True)
speech.rename(columns={'call_id':'ID_LLAMADA'},inplace=True)

In [134]:
### Luego de exportar borro los duplicados y me quedo con el último
data=data.sort_values(['IDENT_CLIENTE','sequence'],ascending=True).drop_duplicates('IDENT_CLIENTE',keep='last')
data.filter(like='ESTADO_ACC').columns
# cols.insert(0,'Efectivo').insert(0,'Fallidas').insert(0,'Exitosas')
c=data[['ID_LLAMADA','Regional','Efectivo','Fallidas','Exitosas','Agrupación','CIUDAD_DPTO','SERVICIO_1', 'PLAN_SERVICIO_1',
       'SERVICIO_2', 'PLAN_SERVICIO_2', 'SERVICIO_3', 'PLAN_SERVICIO_3','MOTIVO','RESULTADO_SS','MT','Segmento','SERVICIOS',
       'ESTADO_ACCION_TO', 'ESTADO_ACCION_TV', 'ESTADO_ACCION_BA',
       'ESTADO_ACCION_CP', 'ESTADO_ACCION_TO_2', 'ESTADO_ACCION_TV_2',
       'ESTADO_ACCION_BA_2','BENEF_MOVIL', 'BENEF_TO', 'BENEF_TV', 'BENEF_BA','Efectividad','Zona','SERVICIOS_2','sequence','serv_dir','accion',
       ]]

In [135]:
merge1=pd.merge(speech,c,on='ID_LLAMADA',how='inner')

In [136]:
merge1.shape

(5726, 61)

In [137]:
speech.filter(like='seg').columns
merge2=pd.merge(speech[~speech['ID_LLAMADA'].isin(merge1['ID_LLAMADA'])],c,right_on='ID_LLAMADA',left_on='UcidXSegmento',how='left')

In [138]:
merge2.drop(['ID_LLAMADA_y'],axis=1,inplace=True)

In [139]:
merge2.rename(columns={'ID_LLAMADA_x':'ID_LLAMADA'},inplace=True )

In [140]:
set(merge2.columns) - set(merge1.columns)

set()

In [141]:
set(merge1.columns)-set(merge2.columns)

set()

In [142]:
merge=pd.concat([merge1,merge2])

In [143]:
merge=merge[~merge.duplicated(keep='last')]

In [144]:
# data[data['IDENT_CLIENTE']=='1100392538']

In [145]:
# speech[~(speech['ID_LLAMADA'].isin(merge['ID_LLAMADA']))]  ###  (speech['UcidXSegmento']=='00001074081631286464')

In [146]:
# print(data['ID_LLAMADA'].sample(110).to_list())

In [147]:
speech.shape, merge.shape

((21689, 27), (21687, 61))

In [149]:
merge.to_csv(r'C:\Users\scadacat\Desktop\Retención_2\consolidado.csv',sep=';',encoding='utf-8',index=True)

In [150]:
# doc=pd.read_excel(r"C:\Users\scadacat\Desktop\Retención_2\BDU\Seguimiento Efectividad diaria Octubre 2021 (Con BD).xlsx",dtype=str,sheet_name='BD UNIFICADA')

In [151]:
# doc2=pd.read_excel(r"C:\Users\scadacat\Desktop\Retención_2\Seguimiento Efectividad diaria Septiembre 2021 (Con BD).xlsx",dtype=str,sheet_name='BD UNIFICADA')

In [95]:
#########################################################               LEO DOCUMENTALES                   ###################################################
doc=pd.read_excel(r"C:\Users\scadacat\Desktop\Retención_2\Seguimiento Efectividad diaria Octubre 2021 (Con BD).xlsx",dtype=str,sheet_name='BD UNIFICADA')
doc['FE_REGISTRO']=pd.to_datetime(doc['FE_REGISTRO'].astype(str).str[:10],format="%Y-%m-%d")
doc=doc[doc['ORIGEN'].isin(['N1','NIVEL2'])]
doc=doc[doc['MES']!='MAYO']


In [96]:
dic={'Junio':'06','Julio':'07','Agosto':'08','Septiembre':'09','Octubre':'10'}

In [97]:
doc=doc.sort_values(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN'],ascending=True)
doc.loc[doc['IDENT_CLIENTE'].str.len() < 7, "IDENT_CLIENTE"] = np.nan  
doc.dropna(subset=['IDENT_CLIENTE'],inplace=True)
doc['IDENT_CLIENTE']=doc['IDENT_CLIENTE'].astype(str)
doc['ano']=doc['FE_REGISTRO'].dt.year.astype(str)
doc['mes']=doc['FE_REGISTRO'].dt.month.astype(str)
doc.loc[doc['IDENT_CLIENTE']=='900092385', "IDENT_CLIENTE"] = np.nan  
doc.loc[doc['IDENT_CLIENTE']=='***PRIVADO****', "IDENT_CLIENTE"] = np.nan  
doc.loc[doc['IDENT_CLIENTE']=='*******', "IDENT_CLIENTE"] = np.nan  
doc.dropna(subset=['IDENT_CLIENTE'],inplace=True) 
doc=doc[~doc['IDENT_CLIENTE'].str.contains('[*]',na=False)]     

In [98]:
doc=doc[(doc['Acum. Exitosas'].astype(int)+doc['Acum. Fallidas'].astype(int))>1]

In [99]:
# cant_cant=doc.groupby('IDENT_CLIENTE',as_index=False).agg({'IDENT_CLIENTE':'count'}).rename(columns={'ID_LLAMADA':'cantidad_IDs_llamada'})
# doc =\
# pd.merge(doc,cant_cant,on="IDENT_CLIENTE")
# col         = 'cantidad_IDs_llamada'
# conditions  = [ doc[col] >= 2, (doc[col]==1)]
# choices     = [ "Reincidente", 'único']
    
# doc["Reincidente_BD_IDC"] = np.select(conditions, choices, default=np.nan)

In [100]:
doc=doc[~doc.duplicated()]

In [101]:
grupo=doc.groupby(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN']).agg({'ORIGEN':'count'}).rename(columns={'ORIGEN':'conteo'}).reset_index().\
    sort_values(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN'])
grupo['sequence']=(grupo.sort_values(by=['IDENT_CLIENTE','FE_REGISTRO','ORIGEN'], ascending=True).
                    groupby(['IDENT_CLIENTE','FE_REGISTRO'])).cumcount()+1
grupo['sequence2']=(grupo.sort_values(by=['IDENT_CLIENTE','FE_REGISTRO'], ascending=True).
                            groupby(['IDENT_CLIENTE'])).cumcount()+1

In [102]:
# grupo.groupby(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN']).agg({'ORIGEN':'count'}).rename(columns={'ORIGEN':'conteo'}).reset_index().describe()

In [103]:
conteos=doc.groupby(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN']).agg({'ORIGEN':'count'}).rename(columns={'ORIGEN':'conteo'}).reset_index()
Cantidad_n1=conteos[(conteos['ORIGEN']=='N1')].groupby(['IDENT_CLIENTE'])['conteo'].sum().reset_index(name='N1')
Cantidad_n2=conteos[(conteos['ORIGEN']=='NIVEL2')].groupby(['IDENT_CLIENTE'])['conteo'].sum().reset_index(name='NIVEL2')
documentales=pd.merge(grupo,Cantidad_n1,on='IDENT_CLIENTE').merge(Cantidad_n2,on='IDENT_CLIENTE') ## concateno 

In [104]:
data_doc_agrupado=doc.sort_values(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN']).groupby(['IDENT_CLIENTE'])['ORIGEN'].apply(lambda x: '_'.join(x)).reset_index()
consolidado=pd.merge(documentales,data_doc_agrupado,on='IDENT_CLIENTE').sort_values(['IDENT_CLIENTE','FE_REGISTRO','sequence2']).drop_duplicates('IDENT_CLIENTE',keep='last')

In [105]:
buenos=consolidado[consolidado['N1']>=consolidado['NIVEL2']]
negativo=consolidado[consolidado['N1']<consolidado['NIVEL2']]
negativo['N1#']=negativo.apply(
            lambda row: row['NIVEL2']  if ((row['NIVEL2']> row['N1']))
            else row['NIVEL2'],axis=1)
buenos['N1#']=buenos['N1']
consolidados=pd.concat([negativo,buenos])
consolidado.shape[0]==consolidados.shape[0]

C:\Users\scadacat\AppData\Local\Temp/ipykernel_14288/1130573990.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negativo['N1#']=negativo.apply(
C:\Users\scadacat\AppData\Local\Temp/ipykernel_14288/1130573990.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buenos['N1#']=buenos['N1']


True

In [106]:
#consolidados.head(1)
# doc[doc['IDENT_CLIENTE']=='1000001647']
# documentales.sort_values(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN']).groupby(['IDENT_CLIENTE'])['ORIGEN'].apply(lambda x: '_'.join(x)).reset_index()
# consolidados.head()

In [107]:
consolidados['salida2']=np.where((consolidados['NIVEL2']>=2),'Previamente en N2','Nuevo en N2')

In [108]:
# consolidados['salida2'].value_counts(True)
# doc[doc['IDENT_CLIENTE']=='1000083837']
# consolidados[consolidados['ORIGEN_y'].str.contains('NIVEL2_NIVEL2')]
# consolidados[(consolidados['salida2']=='Nuevo en N2') &(consolidados['ORIGEN_y']=='_NIVEL2') ]
# consolidados['N1#']=consolidados.apply(
#             lambda row: row['NIVEL2'] if (row['NIVEL2']> row['N1#'])
#             else row['N1#'],axis=1)
# consolidados[(consolidados[col]>consolidados[col2])]

In [109]:
col         = 'N1#'
col2        = 'NIVEL2'
condi = [(consolidados[col]>consolidados[col2]),
            (consolidados[col2]==consolidados[col])]
seg_     = [ "Mayoría en N1", 'Misma llegada a n2'] 
consolidados["niveles"] = np.select(condi, seg_, default='ver')
print(consolidados["niveles"].value_counts())
consolidados.sort_values(['IDENT_CLIENTE','FE_REGISTRO'],ascending=True,inplace=True)
consolidados.drop_duplicates('IDENT_CLIENTE',keep='last',inplace=True)

Misma llegada a n2    58500
Mayoría en N1         36558
Name: niveles, dtype: int64


In [152]:
dic={'100':'Retenidos todos los servicios',
    '33':'Retenidos 1/3 servicios',
    '67':'Retenidos 2/3 servicios',
    '50':'Retenidos 1/2 servicios',
    '0':'No se retuvo ningun servicio'
        }
merge['Efectividad']=merge['Efectividad'].astype(str)
merge['Efectividad']=merge['Efectividad'].replace(dic)

In [153]:
merge['Efectividad'].fillna('No ubicado',inplace=True )

In [154]:
merge['Efectividad'].unique()

array(['100.0', '0.0', '33.0', '50.0', '67.0', 'nan'], dtype=object)

In [155]:
dic={'100.0':'Retenidos todos los servicios',
    '33.0':'Retenidos 1/3 servicios',
    '67.0':'Retenidos 2/3 servicios',
    '50.0':'Retenidos 1/2 servicios',
    '0.0':'No se retuvo ningun servicio'
        }
merge['Efectividad']=merge['Efectividad'].astype(str)
merge['Efectividad']=merge['Efectividad'].replace(dic)

In [156]:
# merge.shape

(21687, 61)

In [157]:
merge.to_csv(r'Y:\01 Informes_Analitica\Retencion\BD_retencion\consolidado2_flujo_v2(sindoc).csv',sep=';',encoding='utf-8',index=False)

In [53]:
merge.head()

,UcidXSegmento,Programa,Topics,Agent,Duracion,Categories,ID_LLAMADA,codigo_finalizacion,login_id,segment_id,Fecha_Grab,Mes,Fecha,Viaje,TopicoFinal,Cat_Sac,Agrupamiento2,TopicoFinal_01,TopicoFinal_02,TopicoFinal_03,TopicoFinal_04,TopicoFinal_05,TopicoFinal_06,TopicoFinal_07,TopicoFinal_08,TopicoFinal_09,TopicoFinal_10,TopicoFinal_11,TopicoFinal_12,Regional,Efectivo,Fallidas,Exitosas,Agrupación,CIUDAD_DPTO,SERVICIO_1,PLAN_SERVICIO_1,SERVICIO_2,PLAN_SERVICIO_2,SERVICIO_3,PLAN_SERVICIO_3,MOTIVO,RESULTADO_SS,MT,Segmento,SERVICIOS,ESTADO_ACCION_TO,ESTADO_ACCION_TV,ESTADO_ACCION_BA,ESTADO_ACCION_CP,ESTADO_ACCION_TO_2,ESTADO_ACCION_TV_2,ESTADO_ACCION_BA_2,BENEF_MOVIL,BENEF_TO,BENEF_TV,BENEF_BA,Efectividad,Zona,SERVICIOS_2,sequence,serv_dir,accion
0,00001000081628696001,UNE_FIDELI_NACIONAL,inicia tiempo de espera,7849306 luis eduardo castaneda za,"743,46",Home| No linguisticos| Silencio 30% a 40%| Cat...,00001040221628695151,no asignado,lcastaza,00001040221628695151,2021-08-11T15:33:21Z,agosto,2021-08-11,NaN,tiempos de espera,tópicos_generales,NaN,inicia tiempo de espera,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOROCCIDENTE,Retenido,0,3,Competencia,ITAGUI-ANTIOQUIA,INTERNET,5 MEGAS NET TRIO,TELEFONIA,ILIMITADO LOCAL,TELEVISION,ESENCIAL TRIO,Sustituyo el producto TO|TV|BA,Desfavorable,NO,Master,INTERNET_TELEFONIA_TELEVISION,Exitosa,Exitosa,Exitosa,0,TO,TV,BA,NaN,BCCO (Cliente lo va consultar x Oferta N2),BCCO (Cliente lo va consultar x Oferta N2),BCCO (Cliente lo va consultar x Oferta N2),100.0,URBANA,INTERNET_TELEFONIA_TELEVISION,1.0,1.0,RETENCION BA_TO_TV
1,00001000111630005441,UNE_FIDELI_NACIONAL,finaliza tiempo de espera,7854375 ricardo andres alzate uri,"637,30",Home| No linguisticos| Silencio 20% a 30%| Cat...,00001052471630004573,no asignado,ralzateu,00001052471630004573,2021-08-26T19:17:21Z,agosto,2021-08-26,NaN,tiempos de espera,tópicos_generales,NaN,finaliza tiempo de espera,retirar servicio por situacion actual,solicitud retiro parcial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSTA,Retenido,0,3,Precio,Galapa-ATLANTICO,INTERNET,Internet 30MB Trio,TELEFONIA,TO ILIMITADO 1.0 TRIO,TELEVISION,CLASICA HD TRIO,Situacion economica desmejoro,Desfavorable,NO,Young,INTERNET_TELEFONIA_TELEVISION,Exitosa,Exitosa,Exitosa,0,TO,TV,BA,NaN,BEF (Explicacion Factura),BEF (Explicacion Factura),BEF (Explicacion Factura),100.0,RURAL,INTERNET_TELEFONIA_TELEVISION,1.0,1.0,RETENCION BA_TO_TV
2,00001000111630005441,UNE_FIDELI_NACIONAL,retirar servicio por situacion actual,7854375 ricardo andres alzate uri,"637,30",Home| No linguisticos| Silencio 20% a 30%| Cat...,00001052471630004573,no asignado,ralzateu,00001052471630004573,2021-08-26T19:17:21Z,agosto,2021-08-26,NaN,retirar servicio por situación actual,motivos económicos,motivos economicos,finaliza tiempo de espera,retirar servicio por situacion actual,solicitud retiro parcial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSTA,Retenido,0,3,Precio,Galapa-ATLANTICO,INTERNET,Internet 30MB Trio,TELEFONIA,TO ILIMITADO 1.0 TRIO,TELEVISION,CLASICA HD TRIO,Situacion economica desmejoro,Desfavorable,NO,Young,INTERNET_TELEFONIA_TELEVISION,Exitosa,Exitosa,Exitosa,0,TO,TV,BA,NaN,BEF (Explicacion Factura),BEF (Explicacion Factura),BEF (Explicacion Factura),100.0,RURAL,INTERNET_TELEFONIA_TELEVISION,1.0,1.0,RETENCION BA_TO_TV
3,00001000111630005441,UNE_FIDELI_NACIONAL,solicitud retiro parcial,7854375 ricardo andres alzate uri,"637,30",Home| No linguisticos| Silencio 20% a 30%| Cat...,00001052471630004573,no asignado,ralzateu,00001052471630004573,2021-08-26T19:17:21Z,agosto,2021-08-26,NaN,solicitud retiro parcial,solicitud retiro parcial,no uso del servicio,finaliza tiempo de espera,retirar servicio por situacion actual,solicitud retiro parcial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSTA,Retenido,0,3,Precio,Galapa-ATLANTICO,INTERNET,Internet 30MB Trio,TELEFONIA,TO ILIMITADO 1.0 TRIO,TELEVISION,CLASICA HD TRIO,Situacion economica desmejoro,Desfavorable,NO,Young,INTERNET_TELEFONIA_TELEVISION,Exitosa,Exitosa,Exitosa,0,TO,TV,BA,NaN,

In [63]:
todo=pd.merge(merge,consolidados,right_on='IDENT_CLIENTE',left_on='CedulaCliente',how='left')
todo.rename(columns={'sequence2':'Nivel de esfuerzo'},inplace=True)
dic={'100':'Retenidos todos los servicios',
    '33':'Retenidos 1/3 servicios',
    '67':'Retenidos 2/3 servicios',
    '50':'Retenidos 1/2 servicios',
    '0':'No se retuvo ningun servicio'
        }
todo['Efectividad']=todo['Efectividad'].astype(str)
todo['Efectividad']=todo['Efectividad'].replace(dic)
todo['class_NE']=  pd.cut(x=todo['Nivel de esfuerzo'], bins=[0,1,2,3,6,100], labels=["1","2","3","4-6",">7"])

KeyError: 'CedulaCliente'

In [ ]:
todo['Fecha']=pd.to_datetime(todo['Fecha'],infer_datetime_format=True)

In [ ]:
# todo[todo['Topics'].str.contains('viaj',na=False)]
# todo[todo['Topics'].isin(['Viaje temporal del usuario']]
# todo[(todo['Fecha']>='2021-10-01') & (todo['Topics'].isin(['Viaje temporal del usuario','viaje definitivo del país',
#                                                         'entrega del inmueble por viaje',
#                                                         'cambio de domicilio','cambio de ciudad']))]
# todo[(odo['Fecha']>='2021-10-01'])['Topics'].value_counts(dropna=False)

In [ ]:
### Visualualización de topicos para el mes de octubre
todo[(todo['MOTIVO']=='Viaje') & (todo['Fecha']>='2021-10-01')]['Topics'].value_counts(dropna=False)

tiempos de espera               10
NaN                              3
venta o entrega del inmueble     2
solicitud retiro parcial         1
retiro por no cobertura          1
intermitencia del wifi           1
Name: Topics, dtype: int64

In [ ]:
# todo['MOTIVO'].isna().sum()/speech.shape[0]
# todo['MOTIVO'].value_counts(dropna=False)

0.7002605040673795

In [ ]:
## Base de datos final
# todo.to_csv(r'C:\Users\scadacat\Desktop\Retención_2\consolidado2_flujo_v2.csv',sep=';',encoding='utf-8',index=True)
todo.to_csv(r'Y:\01 Informes_Analitica\Retencion\BD_retencion\consolidado2_flujo_v2.csv',sep=';',encoding='utf-8',index=False)

In [106]:
# todo[todo['CedulaCliente']=='1000533756']
# bd[bd['IDENT_CLIENTE']=='63466389']
# todo[(todo['salida2']=='Previamente en N2')  & (todo['NIVEL2']>todo['N1'])]
# todo[todo['salida2']=='Nuevo en N2']
# pd.crosstab(todo['NIVEL2'],todo['Efectividad'],normalize='index')
todo['Efectividad'].value_counts()

## Fin del notebook

In [99]:
cant_n1=N1.groupby(['IDENT_CLIENTE'])['sequence'].sum()

In [84]:
grupo[grupo['ORIGEN']=='N1']

,IDENT_CLIENTE,FE_REGISTRO,ORIGEN,conteo,sequence,sequence2
0,1000000908,2021-06-25,N1,1,1,1
2,1000001647,2021-07-14,N1,2,1,1
4,1000001647,2021-07-15,N1,1,1,3
7,1000001720,2021-07-18,N1,1,1,1
8,1000001720,2021-07-21,N1,1,1,2
...,...,...,...,...,...,...
434830,9990664,2021-08-19,N1,1,1,1
434832,9990664,2021-08-23,N1,1,1,3
434839,9992181,2021-08-03,N1,1,1,1
434840,9993310,2021-06-22,N1,1,1,1


In [82]:
grupo.groupby(['IDENT_CLIENTE','ORIGEN'])

,IDENT_CLIENTE,FE_REGISTRO,ORIGEN,conteo,sequence,sequence2
0,1000000908,2021-06-25,N1,1,1,1
1,1000000908,2021-06-25,NIVEL2,1,2,2
2,1000001647,2021-07-14,N1,2,1,1
3,1000001647,2021-07-14,NIVEL2,2,2,2
4,1000001647,2021-07-15,N1,1,1,3


In [ ]:
merge1=merge1.sort_values(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN'],ascending=True)


In [ ]:
grupo=merge1.groupby(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN']).agg({'ORIGEN':'count'}).rename(columns={'ORIGEN':'conteo'}).reset_index().\
    sort_values(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN'])
grupo['sequence']=(grupo.sort_values(by=['IDENT_CLIENTE','FE_REGISTRO','ORIGEN'], ascending=True).
                    groupby(['IDENT_CLIENTE','FE_REGISTRO'])).cumcount()+1
grupo['sequence2']=(grupo.sort_values(by=['IDENT_CLIENTE','FE_REGISTRO'], ascending=True).
                            groupby(['IDENT_CLIENTE'])).cumcount()+1

In [63]:
doc.sort_values(['IDENT_CLIENTE','FE_REGISTRO']).head(10)

,FILIAL,CRM,MES,IDENT_CLIENTE,FE_REGISTRO,USER_REG,ID_REGISTRO,ORIGEN,MOTIVOS HOMOLOGADOS,HERRAMIENTA DE RETENCION,DETALLE HERRAMIENTA DE RETENCION,HERRAMIENTA HOMOLOGADA,Acum. Exitosas,Acum. Fallidas,Accion sin reventa,No Pasa Confronta,No atendido/Cliente Retirado,Retiro Rechazado,CIUDAD DPTO,ano,mes
377402,TIGO,IDCN1,JUNIO,1000000908,2021-06-25,acastape,4274695,N1,COMPETENCIA,SIN HERRAMIENTA,SIN HERRAMIENTA,NaN,0,3,0,0,0,0,Bogota D.C.-Distrito Capital De Bogota,2021,6
380664,TIGO,IDC,JUNIO,1000000908,2021-06-25,vbarrien,99992531521,NIVEL2,COMPETENCIA,No corresponde a una retención,NaN,NaN,0,0,2,0,0,0,NaN,2021,6
498735,TIGO,IDCN1,JULIO,1000001647,2021-07-14,jgranads,4364711,N1,SERVICIO,BENEFICIO,Beneficios de estar con Tigo Hogar,NaN,0,2,0,0,0,0,Bogota D.C.-Distrito Capital De Bogota,2021,7
499479,TIGO,IDCN1,JULIO,1000001647,2021-07-14,ahenaoal,4366496,N1,SERVICIO,SIN HERRAMIENTA,SIN HERRAMIENTA,NaN,0,3,0,0,0,0,Bogota D.C.-Distrito Capital De Bogota,2021,7
501880,TIGO,IDC,JULIO,1000001647,2021-07-14,kcastrog,99992580746,NIVEL2,SERVICIO,Beneficio,NaN,NaN,3,0,0,0,0,0,NaN,2021,7
502641,TIGO,IDC,JULIO,1000001647,2021-07-14,mzuluaa,99992581561,NIVEL2,ATENCION AL CLIENTE,Beneficio,NaN,NaN,3,0,0,0,0,0,NaN,2021,7
507238,TIGO,IDCN1,JULIO,1000001647,2021-07-15,ymanzani,4371134,N1,SERVICIO,SIN HERRAMIENTA,SIN HERRAMIENTA,NaN,0,3,0,0,0,0,Bogota D.C.-Distrito Capital De Bogota,2021,7
510281,TIGO,IDC,JULIO,1000001647,2021-07-15,jmunz,99992584372,NIVEL2,SERVICIO,No corresponde a una retención,NaN,NaN,0,3,0,0,0,0,NaN,2021,7
564682,TIGO,IDC,JULIO,1000001650,2021-07-23,agarrami,99992606878,NIVEL2,FACTURACION,No corresponde a una retención,NaN,NaN,0,0,0,0,0,0,NaN,2021,7
528443,TIGO,IDCN1,JULIO,1000001720,2021-07-18,dcorralc,4386036,N1,SERVICIO,SIN HERRAMIENTA,SIN HERRAMIENTA,NaN,0,3,0,0,0,0,Bogota D.C.-Distrito Capital De Bogota,2021,7


In [80]:
doc.sort_values(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN'],ascending=[False,True,False]).groupby(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN']).agg({'ORIGEN':'count'}).rename(columns={'ORIGEN':'conteo'}).reset_index().\
    sort_values(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN'])

,IDENT_CLIENTE,FE_REGISTRO,ORIGEN,conteo
0,1000000908,2021-06-25,N1,1
1,1000000908,2021-06-25,NIVEL2,1
2,1000001647,2021-07-14,N1,2
3,1000001647,2021-07-14,NIVEL2,2
4,1000001647,2021-07-15,N1,1
...,...,...,...,...
434847,9994066,2021-08-13,NIVEL2,1
434848,99999999902802,2021-08-14,NIVEL2,1
434849,99999999908282,2021-08-25,NIVEL2,1
434850,99999999909314,2021-07-14,NIVEL2,1


In [79]:
# doc.sort_values(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN'],ascending=[True,False,True])

In [43]:
merge1=doc.sort_values(['IDENT_CLIENTE','FE_REGISTRO','ORIGEN'],ascending=True)

In [ ]:
grupos=grupo[(grupo['ORIGEN']=='NIVEL2')]
grupos['salida2']=np.where((grupos['ORIGEN']=='NIVEL2') & (grupos['sequence2']>1),'Previamente en N2','Nuevo en N2')